[Документация](https://docs.aiohttp.org/en/stable/)

[public api](https://github.com/public-apis/public-apis)

In [2]:
import asyncio
from dataclasses import dataclass

import aiohttp

In [3]:
import nest_asyncio
nest_asyncio.apply()

In [14]:
faker_url = 'https://fakerapi.it/api/v2/users?_quantity=1'

## Сессия

In [15]:
async with aiohttp.ClientSession(
    raise_for_status=True,  # Автоматическая обработка ошибок
) as session:
    async with session.get(faker_url) as response:
        res = await response.json()

print(res['data'])

[{'id': 1, 'uuid': 'd1ea43bf-0c53-327c-8638-6c0f67cc676a', 'firstname': 'Zoe', 'lastname': 'Larson', 'username': 'fokeefe', 'password': 'MXauk9X', 'email': 'brisa88@yahoo.com', 'ip': '7.75.206.178', 'macAddress': '3E:CC:AB:B3:C7:A3', 'website': 'http://torphy.org', 'image': 'http://placeimg.com/640/480/people'}]


## headers, params, cookies

In [27]:
async with aiohttp.ClientSession(headers={'Accept': 'application/json'}) as session:
    async with session.get(faker_url, cookies={'foo': 'bar'}, params={'q': 'search'}) as response:
        result = await response.json()

print(result['data'])

[{'id': 1, 'uuid': '13726011-6472-3d1a-8f5c-dd7427e40cbc', 'firstname': 'Zella', 'lastname': 'Adams', 'username': 'mathilde.anderson', 'password': 'dn|;tMQR\\4Vng*+g(r', 'email': 'phoebe.stanton@hotmail.com', 'ip': '3.64.147.241', 'macAddress': '55:AC:D1:21:ED:3D', 'website': 'http://donnelly.net', 'image': 'http://placeimg.com/640/480/people'}]


## get, post, put, delete

In [ ]:
async with aiohttp.ClientSession() as session:
    async with session.get(faker_url) as response:
        await response.json()

    async with session.post(faker_url, json={'foo': 'bar'}) as response:
        await response.json()

    async with session.put(faker_url, json={'foo': 'bar'}) as response:
        await response.json()

    async with session.delete(faker_url) as response:
        await response.json()

## Таймауты

In [ ]:
timeout = aiohttp.ClientTimeout(
    total=15,
    connect=5,
    sock_connect=5,
    sock_read=10,
)

async with aiohttp.ClientSession(timeout=timeout) as session:
    async with session.get(faker_url) as response:
        await response.json()

## Обработка ошибок

In [34]:
async with aiohttp.ClientSession() as s:
        
    try:
        async with s.get('🤨' + faker_url) as r:
            r.raise_for_status()
            data = await r.json()
    except aiohttp.ClientResponseError as e:
        print("HTTP error", e.status, e.message)
    except aiohttp.ClientError as e:
        print("Network error", e)
    except asyncio.TimeoutError:
        print("Timeout")


Network error %F0%9F%A4%A8https://fakerapi.it/api/v2/users?_quantity=1


## Примеры

In [4]:
@dataclass
class Service:
    name: str
    url: str

In [ ]:
ip_services = [
    Service('ipify.org', 'https://api.ipify.org?format=json'),
    Service('httpbin.org (https)', 'https://httpbin.org/get'),
]

In [ ]:
from time import time


async def fetch_ip(service):
    start = time()

    print(f'Sending request to {service.name}...', end='')

    async with aiohttp.ClientSession() as session:
        async with session.get(service.url) as response:
            print(response.status)
            result = await response.json()
            
            end = time()

    print(f'Время выполнения запроса ({service.name}): {end - start:.2f} s')

    return result
    

In [7]:
for service in ip_services:
    res = await fetch_ip(service)

    print(res)

Sending request to ipify.org...200
Время выполнения запроса (ipify.org): 0.79 s
{'ip': '213.111.131.113'}
Sending request to httpbin.org...200
Время выполнения запроса (httpbin.org): 3.81 s
{'args': {}, 'headers': {'Accept': '*/*', 'Accept-Encoding': 'gzip, deflate', 'Host': 'httpbin.org', 'User-Agent': 'Python/3.12 aiohttp/3.12.15', 'X-Amzn-Trace-Id': 'Root=1-68cbb841-67d9c90863bfbc37469e7013'}, 'origin': '213.111.131.113', 'url': 'https://httpbin.org/get'}


In [8]:
async def fetch_all(services: list[Service]):
    async with asyncio.TaskGroup() as tg:
        ipify_result = tg.create_task(fetch_ip(services[0]))
        httpbin_result = tg.create_task(fetch_ip(services[1]))
    
    return ipify_result, httpbin_result

In [9]:
import timeit

timeit.timeit(lambda: asyncio.run(fetch_all(ip_services)), number=1)

Sending request to ipify.org...Sending request to httpbin.org...200
Время выполнения запроса (ipify.org): 0.81 s
200
Время выполнения запроса (httpbin.org): 1.82 s


1.8173140159924515